<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/train/mountain/art1131.jpg)|![](./data/validation/forest/cdmc317.jpg)|

The problem is relatively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several different possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pre-trained layers from resnet
4. Report your model's accuracy

## Load in Data

This surprisingly more difficult than it seems, because you are working with directories of images instead of a single file. This boiler plate will help you download a zipped version of the directory of images. The directory is organized into "train" and "validation" which you can use inside an `ImageGenerator` class to stream batches of images thru your model.  


### Download & Summarize the Data

This step is completed for you. Just run the cells and review the results. 

In [0]:
import tensorflow as tf
import os

_URL = 'https://github.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data.zip?raw=true'

path_to_zip = tf.keras.utils.get_file('./data.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'data')

In [0]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [0]:
train_mountain_dir = os.path.join(train_dir, 'mountain')  # directory with our training mountain pictures
train_forest_dir = os.path.join(train_dir, 'forest')  # directory with our training forest pictures
validation_mountain_dir = os.path.join(validation_dir, 'mountain')  # directory with our validation mountain pictures
validation_forest_dir = os.path.join(validation_dir, 'forest')  # directory with our validation forest pictures

In [0]:
num_mountain_tr = len(os.listdir(train_mountain_dir))
num_forest_tr = len(os.listdir(train_forest_dir))

num_mountain_val = len(os.listdir(validation_mountain_dir))
num_forest_val = len(os.listdir(validation_forest_dir))

total_train = num_mountain_tr + num_forest_tr
total_val = num_mountain_val + num_forest_val

In [5]:
print('total training mountain images:', num_mountain_tr)
print('total training forest images:', num_forest_tr)

print('total validation mountain images:', num_mountain_val)
print('total validation forest images:', num_forest_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training mountain images: 254
total training forest images: 270
total validation mountain images: 125
total validation forest images: 62
--
Total training images: 524
Total validation images: 187


### Keras `ImageGenerator` to Process the Data

This step is completed for you, but please review the code. The `ImageGenerator` class reads in batches of data from a directory and pass them to the model one batch at a time. Just like large text files, this method is advantageous, because it stifles the need to load a bunch of images into memory. 

Check out the documentation for this class method: [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class). You'll expand it's use in the third assignment objective.

In [0]:
batch_size = 16
epochs = 25 # Altered for time.
IMG_HEIGHT = 224
IMG_WIDTH = 224
CHNLS = 3

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [8]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),  # Removed CHNLS for correct shape. This will, however, be needed in my custom CNN.
                                                           class_mode='binary')

Found 533 images belonging to 2 classes.


In [9]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),  # Same thing.
                                                              class_mode='binary')

Found 195 images belonging to 2 classes.


## Instatiate Model

In [10]:
import numpy as np
 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API
 
resnet = ResNet50(weights='imagenet', include_top=False)


for layer in resnet.layers:
    layer.trainable = False


x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [0]:
# Compile the model.
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Using binary_crossentropy because this is a binary classification issue.

## Fit Model

In [12]:
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/25
32/32 [==============================] - 100s 3s/step - loss: 0.8536 - accuracy: 0.5391 - val_loss: 0.6546 - val_accuracy: 0.6705
Epoch 2/25
32/32 [==============================] - 95s 3s/step - loss: 0.6154 - accuracy: 0.6587 - val_loss: 0.5637 - val_accuracy: 0.7914
Epoch 3/25
32/32 [==============================] - 95s 3s/step - loss: 0.5325 - accuracy: 0.7764 - val_loss: 0.5776 - val_accuracy: 0.7975
Epoch 4/25
32/32 [==============================] - 96s 3s/step - loss: 0.4870 - accuracy: 0.7924 - val_loss: 0.3780 - val_accuracy: 0.7975
Epoch 5/25
32/32 [==============================] - 96s 3s/step - loss: 0.4785 - accuracy: 0.7745 - val_loss: 0.6891 - val_accuracy: 0.6074
Epoch 6/25
32/32 [==============================] - 96s 3s/step - loss: 0.3991 - accuracy: 0.8323 - val_loss: 0.4583 - val_accuracy: 0.8282
Epoch 7/25
32/32 [==============================] - 96s 3s/step - loss: 0.3647 - accuracy: 0.8862 - val_loss: 0.3389 - val_accuracy: 0.8834
Epoch 8/25
32/32 [=

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [13]:
# Define the Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten

model = Sequential()

model.add(Conv2D(50, (3,3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, CHNLS), name='InputLayer'))
model.add(MaxPooling2D((2,2), name='PoolingLayer1'))
model.add(Conv2D(75, (3,3), activation='relu', name='ConvLayer2'))
model.add(MaxPooling2D((2,2),  name='PoolingLayer2'))
model.add(Flatten())
model.add(Dense(50, activation='relu', name='HiddenDense'))
model.add(Dense(1, activation='sigmoid', name='OutputLayer'))

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
InputLayer (Conv2D)          (None, 222, 222, 50)      1400      
_________________________________________________________________
PoolingLayer1 (MaxPooling2D) (None, 111, 111, 50)      0         
_________________________________________________________________
ConvLayer2 (Conv2D)          (None, 109, 109, 75)      33825     
_________________________________________________________________
PoolingLayer2 (MaxPooling2D) (None, 54, 54, 75)        0         
_________________________________________________________________
flatten (Flatten)            (None, 218700)            0         
_________________________________________________________________
HiddenDense (Dense)          (None, 50)                10935050  
_________________________________________________________________
OutputLayer (Dense)          (None, 1)                 5

In [0]:
# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
# Fit Model
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/25
32/32 [==============================] - 57s 2s/step - loss: 0.7218 - accuracy: 0.7844 - val_loss: 0.3153 - val_accuracy: 0.8977
Epoch 2/25
32/32 [==============================] - 57s 2s/step - loss: 0.1867 - accuracy: 0.9301 - val_loss: 0.1510 - val_accuracy: 0.9375
Epoch 3/25
32/32 [==============================] - 57s 2s/step - loss: 0.1764 - accuracy: 0.9341 - val_loss: 0.4280 - val_accuracy: 0.8098
Epoch 4/25
32/32 [==============================] - 56s 2s/step - loss: 0.0952 - accuracy: 0.9661 - val_loss: 0.2279 - val_accuracy: 0.9141
Epoch 5/25
32/32 [==============================] - 56s 2s/step - loss: 0.1482 - accuracy: 0.9481 - val_loss: 0.1629 - val_accuracy: 0.9264
Epoch 6/25
32/32 [==============================] - 56s 2s/step - loss: 0.0669 - accuracy: 0.9760 - val_loss: 0.3808 - val_accuracy: 0.8896
Epoch 7/25
32/32 [==============================] - 56s 2s/step - loss: 0.0913 - accuracy: 0.9681 - val_loss: 0.1312 - val_accuracy: 0.9325
Epoch 8/25
32/32 [==

# Custom CNN Model with Image Manipulations

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Simply, you should be able to modify our image generator for the problem. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [17]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rotation_range=0,
        width_shift_range=0.25,
        height_shift_range=0.25,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

Using TensorFlow backend.


In [35]:
train_generator = train_datagen.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 533 images belonging to 2 classes.


In [37]:
val_generator = val_datagen.flow_from_directory(batch_size=batch_size,
                                                           directory=validation_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 195 images belonging to 2 classes.


In [39]:
gen = model.fit(
    train_generator,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=total_val // batch_size
)

Epoch 1/25
32/32 [==============================] - 63s 2s/step - loss: 0.2636 - accuracy: 0.9043 - val_loss: 0.2153 - val_accuracy: 0.9318
Epoch 2/25
32/32 [==============================] - 62s 2s/step - loss: 0.2998 - accuracy: 0.8842 - val_loss: 0.8310 - val_accuracy: 0.7239
Epoch 3/25
32/32 [==============================] - 62s 2s/step - loss: 0.1833 - accuracy: 0.9222 - val_loss: 0.3495 - val_accuracy: 0.8528
Epoch 4/25
32/32 [==============================] - 61s 2s/step - loss: 0.2225 - accuracy: 0.9202 - val_loss: 0.3040 - val_accuracy: 0.8405
Epoch 5/25
32/32 [==============================] - 61s 2s/step - loss: 0.2465 - accuracy: 0.8982 - val_loss: 0.3126 - val_accuracy: 0.8405
Epoch 6/25
32/32 [==============================] - 64s 2s/step - loss: 0.1667 - accuracy: 0.9401 - val_loss: 0.2448 - val_accuracy: 0.8896
Epoch 7/25
32/32 [==============================] - 61s 2s/step - loss: 0.1968 - accuracy: 0.9222 - val_loss: 0.7946 - val_accuracy: 0.7178
Epoch 8/25
32/32 [==

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language